In [16]:
#importing all necessary libraries 
import pandas as pd
import numpy as py

#check version
print("Pandas version: " + pd.__version__)


Pandas version: 2.2.2


In [15]:
#load csv file with the correct delimiter 
file_path = 'CTU-IoT-Malware-Capture-34-1conn.log.labeled copy.csv'
df = pd.read_csv(file_path, delimiter='|')

#take a look, displaying the first few rows to verify correct loading 
print("Initial data:") 
print(df.head())



Initial data:
             ts                 uid      id.orig_h  id.orig_p       id.resp_h  \
0  1.545404e+09   CrDn63WjJEmrWGjqf  192.168.1.195      41040  185.244.25.235   
1  1.545404e+09  CY9lJW3gh1Eje4usP6  192.168.1.195      41040  185.244.25.235   
2  1.545404e+09   CcFXLynukEDnUlvgl  192.168.1.195      41040  185.244.25.235   
3  1.545404e+09   CDrkrSobGYxHhYfth  192.168.1.195      41040  185.244.25.235   
4  1.545404e+09  CTWZQf2oJSvq6zmPAc  192.168.1.195      41042  185.244.25.235   

   id.resp_p proto service  duration orig_bytes  ... local_resp missed_bytes  \
0         80   tcp       -  3.139211          0  ...          -            0   
1         80   tcp       -         -          -  ...          -            0   
2         80   tcp       -         -          -  ...          -            0   
3         80   tcp    http  1.477656        149  ...          -         2896   
4         80   tcp       -  3.147116          0  ...          -            0   

       history ori